## **Food Delivery Routing Optimization Solution**



# Context:

---

In last few years the food e-catering business have grown exponentially. This leads to increase in demand for better solutions which involve assigning orders to Delivery Partners optimally. This notebook deals a solution in the similar direction with the given data.


# Porblem: 

---

This problem is an extenstion of Vehicle Routing Problem. The added constraints are the capacity of vehicle, multiple pickup-delivery locations. 

# Solution: 

---

During the last 50+ years a lot of research has been done on solving Vehicle Routing Problem. This notebook deals with one of the solutions which is implemented in the Google *ortools* library.
This problem basically deals with trying out multiple combinations to get the smallest path. 

The alogorithm used in this notebook is local search options which finds an optimal solution. The time complexity is very high as it usually work well for small number of delieveres(upto 15).

Since the data size is very small (20 deliveries) the implemented algorithm is sufficient to find out the optimal results with given constraints on maximum capacity and multiple pickup-deliveries.


# Tools Used: 

---
Google Developer OR Tools, pandas, math

# Assumptions: 

---

1. Since the distance coordinates are given in (lat,lon) format, "Haversine" formula is used to calculate the distance between any two points.

2. To simplify the data the *order_id* are assigned indices in range [1,20] and correspondingly the *rest_id* are assigned indices in range [21, 40]. Any *rest_id* correspond to *order_id + 20*

3. Each restaurant already have at least 1 Delivery Partner. Since this algorithm uses a depot station which initially have all the delivery partners. The Delivery Partner starts from depot -> take order -> deliever -> go back to depot. Since we are dealing with multiple depots and to simplify the problem the ditance from depot to any restaurant or order location is assigned 0 so that it is not taken into consideration. 

4. The solution tries to minimize the maximum distance travelled by any Delivery Partner and then calculates the total distance travelled by every Delivery Partner. The ditance parameter is set to the maximum distance between pickup-delivery location which finally also reduces the total distance travelled by all delievery executives.

Relevant comments have been added with the code snippets.

In [1]:
"""Importing Google ortools and other relevant libraries"""
%pip install ortools
import math
from math import cos, asin, sqrt, pi
import pandas as pd
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


/Users/kartikmodi/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
"""Haversine formula to calculate distance between two points with lat lon given"""
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a))

In [8]:
# """Creating data into relevant format to that of the routing model"""
file_path = './data/sample_data_set_assignment_v6.xlsx'
data = pd.read_excel(file_path)

# Assigning new indices to rest_id and order_id

for i in range(data.shape[0]):
  data.loc[i, "order_id"] = i + 1
  data.loc[i, "rest_id"] = i + 1 + data.shape[0]

# Putting all the location of restaurants and orders in a dictonary to easily calculate distance between any two locations
dict = {}
for i in range(data.shape[0]):
  dict[data.loc[i, "order_id"]] = data.loc[i, "rest_location"]
  dict[data.loc[i, "rest_id"]] = data.loc[i, "cust_location"]

cols = 41
rows = cols
mx = 0

# Creating the distance matrix for distance between all pairs of locations
arr = [[0 for i in range(cols)] for j in range(rows)]
for i in range (1, 41):
  for j in range(1, 41):
    c1 = dict[i].split(',')
    c2 = dict[j].split(',')
    arr[i][j] = math.floor(distance(float(c1[0]), float(c1[1]), float(c2[0]), float(c2[1])) * 1000)
    arr[j][i] = arr[i][j]

# Calculating the maximum distance between all pickup-delivery location
for i in range (1, 21):
  c1 = dict[i].split(',')
  c2 = dict[i+20].split(',')
  mx = max(math.floor(distance(float(c1[0]), float(c1[1]), float(c2[0]), float(c2[1])) * 1000), mx)


In [4]:
# Creating the data model to feed into the pywrapcp RoutingModel routing manager
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = arr
    data['demands'] = [0]*41
    for i in range(1, 21):
      data['demands'][i] = -1;
      data['demands'][i+20] = 1;
    arr2 = [[0 for i in range(2)] for j in range(20)]
    for i in range (0, 20):
      arr2[i][0] = i + 1 + 20;
      arr2[i][1] = i + 1;
    data['pickups_deliveries'] = arr2
    data['vehicle_capacities'] = [3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3]
    data['num_vehicles'] = 20
    data['depot'] = 0
    return data
data = create_data_model()

In [5]:
# Function to print the opitmal paths of every DE and distance travelled with all constraints
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            if(manager.IndexToNode(index) != 0):
              plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            if(manager.IndexToNode(index) != 0):
              route_distance += routing.GetArcCostForVehicle(
                  previous_index, index, vehicle_id)
        # plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))

In [6]:
# Function to initialize the Routing model and calculat the parameters for routing model which minimizes the total cost.
def main():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        mx,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
        routing.solver().Add(distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))

    # Setting local search metaheuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 1
    search_parameters.log_search = True
    # search_parameters.time_limit.seconds = 300

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        print_solution(data, manager, routing, assignment)


In [7]:
# Printing the final paths of all DE's
if __name__ == '__main__':
    main()

I0000 00:00:1708982923.372484 2093472 search.cc:282] Start search (memory used = 76.69 MB)
I0000 00:00:1708982923.372843 2093472 search.cc:282] Root node processed (time = 0 ms, constraints = 491, memory used = 76.86 MB)
I0000 00:00:1708982923.384096 2093472 search.cc:282] Solution #0 (470929, time = 11 ms, branches = 34, failures = 1, depth = 33, memory used = 79.59 MB, limit = 1%)
I0000 00:00:1708982923.390491 2093472 search.cc:282] Solution #1 (470376, maximum = 470929, time = 17 ms, branches = 37, failures = 3, depth = 33, PairRelocateOperator, neighbors = 800, filtered neighbors = 1, accepted neighbors = 1, memory used = 80.70 MB, limit = 1%)
I0000 00:00:1708982923.400883 2093472 search.cc:282] Solution #2 (470197, maximum = 470929, time = 28 ms, branches = 41, failures = 5, depth = 33, PairRelocateOperator, neighbors = 3089, filtered neighbors = 2, accepted neighbors = 2, memory used = 80.81 MB, limit = 2%)
I0000 00:00:1708982923.457749 2093472 search.cc:282] Solution #3 (470393,

I0000 00:00:1708982924.064159 2093472 search.cc:282] Solution #33 (470197, maximum = 470929, time = 691 ms, branches = 224, failures = 112, depth = 33, PairRelocateOperator, neighbors = 217466, filtered neighbors = 33, accepted neighbors = 33, memory used = 80.56 MB, limit = 69%)
I0000 00:00:1708982924.064976 2093472 search.cc:282] Solution #34 (470376, minimum = 470197, maximum = 470929, time = 692 ms, branches = 227, failures = 114, depth = 33, PairRelocateOperator, neighbors = 217610, filtered neighbors = 34, accepted neighbors = 34, memory used = 80.56 MB, limit = 69%)
I0000 00:00:1708982924.106114 2093472 search.cc:282] Solution #35 (470376, minimum = 470197, maximum = 470929, time = 733 ms, branches = 234, failures = 119, depth = 33, PairRelocateOperator, neighbors = 231643, filtered neighbors = 35, accepted neighbors = 35, memory used = 80.58 MB, limit = 73%)
I0000 00:00:1708982924.146719 2093472 search.cc:282] Solution #36 (470376, minimum = 470197, maximum = 470929, time = 774

Route for vehicle 0:
 38 ->  25 ->  18 ->  5 -> Distance of the route: 3172m

Route for vehicle 1:
 35 ->  15 -> Distance of the route: 592m

Route for vehicle 2:
 33 ->  13 -> Distance of the route: 626m

Route for vehicle 3:
 26 ->  6 -> Distance of the route: 1035m

Route for vehicle 4:
 27 ->  37 ->  7 ->  17 -> Distance of the route: 3060m

Route for vehicle 5:
 30 ->  10 -> Distance of the route: 1229m

Route for vehicle 6:
 23 ->  3 -> Distance of the route: 1413m

Route for vehicle 7:
 39 ->  19 -> Distance of the route: 1597m

Route for vehicle 8:
 24 ->  4 -> Distance of the route: 1739m

Route for vehicle 9:
 29 ->  9 -> Distance of the route: 1951m

Route for vehicle 10:
 32 ->  12 -> Distance of the route: 2037m

Route for vehicle 11:
 40 ->  20 -> Distance of the route: 2147m

Route for vehicle 12:
 34 ->  14 -> Distance of the route: 2168m

Route for vehicle 13:
 21 ->  1 -> Distance of the route: 2181m

Route for vehicle 14:
 36 ->  16 -> Distance of the route: 2356m

R

I0000 00:00:1708982924.347125 2093472 search.cc:282] Solution #45 (470393, minimum = 470197, maximum = 470929, time = 974 ms, branches = 299, failures = 157, depth = 33, PairRelocateOperator, neighbors = 317298, filtered neighbors = 45, accepted neighbors = 45, memory used = 80.61 MB, limit = 97%)
I0000 00:00:1708982924.347558 2093472 search.cc:282] Solution #46 (470197, maximum = 470929, time = 974 ms, branches = 304, failures = 159, depth = 33, PairRelocateOperator, neighbors = 317303, filtered neighbors = 46, accepted neighbors = 46, memory used = 80.61 MB, limit = 97%)
I0000 00:00:1708982924.351276 2093472 search.cc:282] Solution #47 (470576, minimum = 470197, maximum = 470929, time = 978 ms, branches = 307, failures = 161, depth = 33, PairRelocateOperator, neighbors = 319146, filtered neighbors = 47, accepted neighbors = 47, memory used = 80.61 MB, limit = 98%)
I0000 00:00:1708982924.351783 2093472 search.cc:282] Solution #48 (470294, minimum = 470197, maximum = 470929, time = 979

## Results:

The minimum distance to be travelled with 20 DE's availabe is 37.797 km. The total cost of travel will be Rs. 113.
The paths of each DE are mentioned above.

Some possible changes to manipulate the results can be:
1. To minimize the total DE's the maximum distance allowed parameter should be increased accordingly. 

2. The capacity of DE's can be changed in individual basis.

3. Time Constraints can also be added as another dimension in the routing model. 

4. The results obtained depends on the time limit the algorithm is allowed to run. The cost usually reduces with more time allowed. The solution presented here works within 1s. The results obtaied when time limit was set upto 60s were also the same.

Author's Comments:

I have treated this Routing model as a Black Box. While there is lot to explore and read about the solutions for such problem, due to time constraints I decided to proceed this way. I am ready to dive deep into the theory behind the algorithm with sufficient time available.

## THANKS!